In [2]:
print("OK")

OK


In [3]:
print("Hello world \n My world is ...")

Hello world 
 My world is ...


In [4]:
import os 
from dotenv import load_dotenv 
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import TokenTextSplitter 

In [8]:
%cd ..

e:\#GenAI_Krish_Nail\7-End-to-End GenAI projects using LLOPS and Deployment\Interview-Question-Creator


In [41]:
# load_dotenv()
# OpenAI_api_key = os.environ.get('OPENAI_API_KEY') 
# OpenAI_api_key

In [6]:
os.environ['OPENAI_API_KEY'] = OpenAI_api_key 

In [7]:
os.getcwd() 

'e:\\#GenAI_Krish_Nail\\7-End-to-End GenAI projects using LLOPS and Deployment\\Interview-Question-Creator\\research'

e:\#GenAI_Krish_Nail\7-End-to-End GenAI projects using LLOPS and Deployment\Interview-Question-Creator


In [9]:
%pwd

'e:\\#GenAI_Krish_Nail\\7-End-to-End GenAI projects using LLOPS and Deployment\\Interview-Question-Creator'

In [10]:
file_path = "data/stats.pdf" 
loader = PyPDFLoader(file_path) 
data = loader.load() 

In [11]:
data

[Document(metadata={'source': 'data/stats.pdf', 'page': 0}, page_content='Importance\nand\nthe\nuse\nof\ncorrelation\nin\nStatistics\nIntroduction\nCorrelation\nis\na\nstatistical\nmeasure\nthat\nexpresses\nthe\nextent\nto\nwhich\ntwo\nvariables\nare\nlinearly\nrelated.\nIt\nis\na\ncommon\ntool\nfor\ndescribing\nsimple\nrelationships\nwithout\nmaking\na\nstatement\nabout\ncause\nand\neffect.\nCorrelation\ncoefficients\nrange\nfrom\n-1\nto\n1,\nwith\na\nvalue\nof\n0\nindicating\nno\nlinear\nrelationship\nbetween\nthe\ntwo\nvariables,\na\nvalue\nof\n1\nindicating\na\nperfect\npositive\nlinear\nrelationship,\nand\na\nvalue\nof\n-1\nindicating\na\nperfect\nnegative\nlinear\nrelationship.\nCorrelation\nis\nimportant\nin\nstatistics\nbecause\nit\ncan\nbe\nused\nto\n1.\nIdentify\nrelationships\nbetween\nvariables:\nCorrelation\ncan\nbe\nused\nto\nidentify\nwhether\nthere\nis\na\nrelationship\nbetween\ntwo\nvariables,\nand\nif\nso,\nwhether\nthe\nrelationship\nis\npositive\nor\nnegative.\nThis

In [12]:
len(data)

3

In [13]:
question_gen = "" 
for page in data: 
    question_gen += page.page_content 

In [14]:
chunking = TokenTextSplitter(

    model_name = 'gpt-3.5-turbo', chunk_size = 10000,
    chunk_overlap = 200 
)

In [15]:
after_chunking = chunking.split_text(question_gen) 

In [16]:
len(after_chunking)  

1

In [17]:
type(after_chunking)

list

# Document Term matrix 

In [18]:
from langchain.docstore.document import Document 

In [19]:
doc_question_after_chunking = [Document (page_content=
                                          x )for x in after_chunking ] 

In [20]:
# print(doc_question_after_chunking)  
print(type(doc_question_after_chunking[0]))  

<class 'langchain_core.documents.base.Document'>


In [21]:
chunking_answer = TokenTextSplitter(

    model_name = 'gpt-3.5-turbo', chunk_size = 1000,
    chunk_overlap = 100 
)

In [22]:
document_answer_gen = chunking_answer.split_documents(doc_question_after_chunking) 

In [23]:
len(document_answer_gen) 

2

# Initializing the LLM model and 

In [24]:
from langchain.chat_models import ChatOpenAI 
from langchain_openai import ChatOpenAI 

In [25]:
llm_ques_gen_pipeline = ChatOpenAI(model = 'gpt-3.5-turbo') 

In [27]:
prompt_template = """

You are an expert at creating questions based on coding materials and documentations. 
Your goal is to prepare a coder or programmer for their exam and coding tests. 
You do this by asking questions about the ext below: 

-------
{text}
-------
Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS: 


"""




In [28]:
from langchain.prompts import PromptTemplate 




In [29]:
PROMPT_QUESTIONS = PromptTemplate(template = prompt_template, input_variable =['text']) 

In [30]:
refine_template = (

    """ 
You are an exper tat creating prcic questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test. 
We have  received some proactice questions to a certain extent: {existing_answer}

------
{text}
------


Given the new context, refine the original questions in English. 
If the context is not helpful, please provide the original questions. 

QUESTIONS: 

    """


)

In [31]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(
input_variable =['existing_answer', 'text' ] ,
template = refine_template 
)

# Creating a chain 

In [32]:
from langchain.chains import load_summarize_chain 

In [33]:
ques_gen_chain = load_summarize_chain (
    llm = llm_ques_gen_pipeline, 
    chain_type = "refine", verbose = False, 
    question_prompt = PROMPT_QUESTIONS, refine_prompt = REFINE_PROMPT_QUESTIONS
    
)

# defining the wrapper 

In [34]:
ques = ques_gen_chain.run(document_answer_gen) 

print(ques)

C:\Users\user\AppData\Local\Temp\ipykernel_21600\3278647007.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ques = ques_gen_chain.run(document_answer_gen)


1. What is correlation and how is it defined in statistics?
2. Explain the range of correlation coefficients and what each value indicates.
3. How can correlation be used to identify relationships between variables?
4. In what ways can correlation be used to make predictions in various fields?
5. Describe how correlation can be used as a starting point for developing causal models.
6. Provide examples of how correlation is used in business, finance, medicine, and psychology.
7. How can correlation be used in the real world by a marketing manager and a financial analyst?
8. Explain the importance of correlation in making better business decisions and reducing risk in finance.
9. How can correlation be used to improve the prevention, diagnosis, and treatment of diseases in medicine?
10. In what ways can correlation be used to develop better psychological assessments and treatments in psychology?


In [35]:
from langchain.embeddings.openai import OpenAIEmbeddings 
# from langchain-openai import OpenAIEmbeddings  

embedding_model = OpenAIEmbeddings() 
from langchain.vectorstores import FAISS 
vectro_store = FAISS.from_documents (document_answer_gen, embedding_model) 

C:\Users\user\AppData\Local\Temp\ipykernel_21600\1542150785.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings()


In [36]:
# Initializing LLM for answer generation 
llm_answer_gen = ChatOpenAI(temperature= 0.1, model = "gpt-3.5-turbo") 

In [37]:

ques_list = ques.split ("\n")


In [38]:
from langchain.chains import RetrievalQA 

In [39]:
answer_generation_chain = RetrievalQA.from_chain_type (llm= llm_answer_gen,  chain_type = "stuff", 
                                                       retriever = vectro_store.as_retriever()) 

In [40]:
# wrapper 

for question in ques_list:
    print("Question", question)
    answer = answer_generation_chain.run(question) 
    print("Answer: ", answer)
    print("------------------------------------------------------------------\\n\n")
    # Save answer to file
    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\n\n")
        f.write("Answer: " + answer + "\\n")
        f.write("-------------------------------------------------------------\\n\n") 

        

Question 1. What is correlation and how is it defined in statistics?
Answer:  Correlation is a statistical measure that expresses the extent to which two variables are linearly related. It is a tool for describing simple relationships without making a statement about cause and effect. Correlation coefficients range from -1 to 1, with 0 indicating no linear relationship, 1 indicating a perfect positive linear relationship, and -1 indicating a perfect negative linear relationship.
------------------------------------------------------------------\n

Question 2. Explain the range of correlation coefficients and what each value indicates.
Answer:  The range of correlation coefficients is from -1 to 1. Here is what each value indicates:

- A correlation coefficient of 0 indicates no linear relationship between the two variables.
- A correlation coefficient of 1 indicates a perfect positive linear relationship, meaning that as one variable increases, the other variable also increases in a li